In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [2]:
import os
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report 
from keras.preprocessing.image import  ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Input
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.applications.vgg16 import VGG16, preprocess_input

In [3]:
train_dir = r"D:\DataSets\Cat_and_Dog\training_set"
test_dir = r"D:\DataSets\Cat_and_Dog\test_set"

In [4]:
train_data=ImageDataGenerator(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        validation_split=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_data=ImageDataGenerator(rescale=1./255)

In [5]:
train_set=train_data.flow_from_directory(directory=train_dir,target_size=(224,224),batch_size=32,color_mode="rgb",class_mode = 'binary',subset='training')
validation_set=train_data.flow_from_directory(directory=train_dir,target_size=(224,224),batch_size=32,color_mode="rgb",class_mode = 'binary',subset='validation')
test_set=test_data.flow_from_directory(directory=test_dir,target_size=(224,224),batch_size=30,shuffle=True,class_mode = 'binary',color_mode="rgb")

Found 6400 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
base_model = VGG16(
    weights = "imagenet", 
    input_shape = (224, 224, 3),
    include_top = False
)


for layers in base_model.layers:
    layers.trainable = False


def vgg16_pretrained():
    
    model = Sequential(
        [
            base_model,
            GlobalAveragePooling2D(),
            Dense(512, activation = "relu"),
            Dense(128, activation = "relu"),
            Dropout(0.4),
            Dense(64, activation = "relu"),
            Dense(1, activation = "sigmoid")
        ]
    )
    
    return model


In [7]:
model = vgg16_pretrained()

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = "accuracy")

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                        

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor = "val_accuracy", 
    patience = 2,
    verbose = 1, 
    factor = 0.5, 
    min_lr = 0.000000001
)

early_stopping = EarlyStopping(
    monitor = "val_accuracy",
    patience = 5,
    verbose = 1,
    mode = "max",
)

checkpoint = ModelCheckpoint(
    monitor = "val_accuracy",
    filepath = "catdog_vgg16_.{epoch:02d}-{val_accuracy:.6f}.hdf5",
    verbose = 1,
    save_best_only = True, 
    save_weights_only = True
)

In [ ]:
history = model.fit_generator(train_set,
    epochs = 10, 
    validation_data = validation_set,
    validation_steps = 1600 // 32,
    steps_per_epoch = 6400 // 32,
#     callbacks = [reduce_lr, early_stopping, checkpoint]
)

Epoch 1/10
200/200 [==============================] - 1619s 8s/step - loss: 0.0000e+00 - accuracy: 0.4988 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 2/10
200/200 [==============================] - 1444s 7s/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 3/10
 71/200 [=========>....................] - ETA: 13:02 - loss: 0.0000e+00 - accuracy: 0.5238

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (12, 4))

sns.lineplot(x = range(len(history.history["loss"])), y = history.history["loss"], ax = axes[0], label = "Training Loss")
sns.lineplot(x = range(len(history.history["loss"])), y = history.history["val_loss"], ax = axes[0], label = "Validation Loss")

sns.lineplot(x = range(len(history.history["accuracy"])), y = history.history["accuracy"], ax = axes[1], label = "Training Accuracy")
sns.lineplot(x = range(len(history.history["accuracy"])), y = history.history["val_accuracy"], ax = axes[1], label = "Validation Accuracy")
axes[0].set_title("Loss"); axes[1].set_title("Accuracy")

sns.despine()
plt.show()

In [ ]:
test_images = os.listdir(D:\DataSets\Cat_and_Dog\test_set)
X_test = pd.DataFrame({
    'test_imagename': test_images
})
samples = X_test.shape[0]

In [ ]:
X_test.count()

In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(samples/batch_size))

In [ ]:
predict.shape,X_test.shape

In [ ]:
# Define treshold
threshold = 0.5
# Convert
Y_pred_conv = np.where(Y_pred > threshold, 1,0)

In [ ]:
Y_pred_conv[:,0]

In [ ]:
Y_pred_conv.dtype

In [ ]:
Y_val.dtype

In [ ]:
# Convert to int
Y_val_str = Y_val.astype(int)